In [1]:
# Check that SparkSession is available
spark

In [2]:
# Start H2OContext in internal backend
from pysparkling import *
hc = H2OContext.getOrCreate(spark)

Connecting to H2O server at http://172.16.2.17:54321... successful.


H2O cluster uptime:,07 secs
H2O cluster version:,3.16.0.1
H2O cluster version age:,6 days
H2O cluster name:,sparkling-water-kuba_local-1512079439108
H2O cluster total nodes:,1
H2O cluster free memory:,3.417 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://172.16.2.17:54321
H2O connection proxy:,None



Sparkling Water Context:
 * H2O name: sparkling-water-kuba_local-1512079439108
 * cluster size: 1
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (driver,172.16.2.17,54321)
  ------------------------

  Open H2O Flow in browser: http://172.16.2.17:54321 (CMD + click in Mac OSX)

    


In [3]:
# Load 
import h2o
reviews = h2o.upload_file("Reviews.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
# Drop unnecessary columns
reviews.drop("Id")
reviews.drop("ProfileName")

Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
1,B001E4KFG0,A3SGXH7AUHU8GW,1,1,5,1.30386e+09,Good Quality Dog Food,I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than most.
2,B00813GRG4,A1D87F6ZCVE5NK,0,0,1,1.34698e+09,Not as Advertised,"Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as """"Jumbo"""
3,B000LQOCH0,ABXLMWJIXXAIN,1,1,4,1.21902e+09,"""""Delight"""" says it a","This is a confection that has been around a few centuries. It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar. And it is a tiny mouthful of heaven. Not too chewy, and very flavorful. I highly recommend this yummy treat. If you are familiar with the story of C.S. Lewis' """"The Lion, The Witch, and The Wardrobe"""" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witc"
4,B000UA0QIQ,A395BORC6FGVXV,3,3,2,1.30792e+09,Cough Medicine,If you are looking for the secret ingredient in Robitussin I believe I have found it. I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda. The flavor is very medicinal.
5,B006K2ZZ7K,A1UQRSCLF8GW1T,0,0,5,1.35078e+09,Great taffy,"Great taffy at a great price. There was a wide assortment of yummy taffy. Delivery was very quick. If your a taffy lover, this is a deal."
6,B006K2ZZ7K,ADT0SRK1MGOEU,0,0,4,1.34205e+09,Nice Taffy,"I got a wild hair for taffy and ordered this five pound bag. The taffy was all very enjoyable with many flavors: watermelon, root beer, melon, peppermint, grape, etc. My only complaint is there was a bit too much red/black licorice-flavored pieces (just not my particular favorites). Between me, my kids, and my husband, this lasted only two weeks! I would recommend this brand of taffy -- it was a delightful treat."
7,B006K2ZZ7K,A1SP2KVKFXXRU1,0,0,5,1.34015e+09,Great! Just as good as the expensive brands!,"This saltwater taffy had great flavors and was very soft and chewy. Each candy was individually wrapped well. None of the candies were stuck together, which did happen in the expensive version, Fralinger's. Would highly recommend this candy! I served it at a beach-themed party and everyone loved it!"
8,B006K2ZZ7K,A3JRGQVEQN31IQ,0,0,5,1.336e+09,"Wonderful, tasty taffy",This taffy is so good. It is very soft and chewy. The flavors are amazing. I would definitely recommend you buying it. Very satisfying!!
9,B000E7L2R4,A1MZYO9TZK0BBI,1,1,5,1.32201e+09,Yay Barley,Right now I'm mostly just sprouting this so my cats can eat the grass. They love it. I rotate it around with Wheatgrass and Rye too
10,B00171APVA,A21BT40VZCCYT4,0,0,5,1.35121e+09,Healthy Dog Food,This is a very healthy dog food. Good for their digestion. Also good for small puppies. My dog eats her required amount at every feeding.


In [5]:
# Refine date column
def refine_date_col(data, col, drop):
    data["Day"] = data[col].day()
    data["Month"] = data[col].month()
    data["Year"] = data[col].year()
    data["WeekNum"] = data[col].week()
    data["WeekDay"] = data[col].dayOfWeek()
    data["HourOfDay"] = data[col].hour()
    
    # Create weekend and season cols
    # Spring = Mar, Apr, May. Summer = Jun, Jul, Aug. Autumn = Sep, Oct. Winter = Nov, Dec, Jan, Feb.
    # data["Weekend"]   = [1 if x in ("Sun", "Sat") else 0 for x in data["WeekDay"]]
    data["Weekend"] = ((data["WeekDay"] == "Sun") | (data["WeekDay"] == "Sat"))
    data["Season"] = data["Month"].cut([0, 2, 5, 7, 10, 12], ["Winter", "Spring", "Summer", "Autumn", "Winter"])
    if drop:
        data.drop(col)
    
refine_date_col(reviews, "Time", True)


In [6]:
# Convert data to spark Data Frame
df = hc.as_spark_frame(reviews)
# Show the schema
df.printSchema()

root
 |-- Id: integer (nullable = false)
 |-- ProductId: string (nullable = false)
 |-- UserId: string (nullable = false)
 |-- ProfileName: string (nullable = false)
 |-- HelpfulnessNumerator: short (nullable = false)
 |-- HelpfulnessDenominator: short (nullable = false)
 |-- Score: byte (nullable = false)
 |-- Time: integer (nullable = false)
 |-- Summary: string (nullable = false)
 |-- Text: string (nullable = false)
 |-- Day: byte (nullable = false)
 |-- Month: byte (nullable = false)
 |-- Year: short (nullable = false)
 |-- WeekNum: byte (nullable = false)
 |-- WeekDay: string (nullable = false)
 |-- HourOfDay: byte (nullable = false)
 |-- Weekend: byte (nullable = false)
 |-- Season: string (nullable = false)



In [7]:
from pyspark.sql.functions import *
# Calculate average score per Year
avg_score_per_year = hc.as_h2o_frame(df.groupBy("Year").agg(mean("Score"), count("Score")), "avgScorePerYear")

In [8]:
# Calculate average score per Month
avg_score_per_month = hc.as_h2o_frame(df.groupBy("Month").agg(mean("Score"), count("Score")), "avgScorePerMonth")

In [9]:
# Calculate average score per Day
avg_score_per_month = hc.as_h2o_frame(df.groupBy("WeekDay").agg(mean("Score"), count("Score")), "avgScorePerDay")

In [11]:
# Calculate input data frame for sentiment analysis
sentimentDF = hc.as_spark_frame(reviews[["Score", "Month", "Day", "WeekDay", "Summary"]])

In [ ]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col, udf
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, IDF, HashingTF


# Prepare data for the pipeline
toFloat = udf(lambda score: float(score), FloatType())
sentimentDF.where("Score != 3").withColumn("Score", toFloat(col("Score")))

# Define the pipeline stages
regexTokenizer = RegexTokenizer(inputCol="Summary",
                                outputCol="TokenizedSummary",
                                pattern="[, ]",
                                toLowercase=True)

stopWordsRemover = StopWordsRemover(inputCol=regexTokenizer.getOutputCol(),
                                    outputCol="CleanedSummary",
                                    caseSensitive=False)
## Hash the words
hashingTF = HashingTF(inputCol=stopWordsRemover.getOutputCol(),
                      outputCol="wordToIndex",
                      numFeatures=1 << 10)


## Create inverse document frequencies model
idf = IDF(inputCol=hashingTF.getOutputCol(),
          outputCol="tf_idf",
          minDocFreq=1)

from pysparkling.ml import ColumnPruner, H2OGBM

## Create GBM model
gbm = H2OGBM(ratio=0.8,
             featuresCols=[idf.getOutputCol()],
             predictionCol="Summary")

## Remove all helper columns
colPruner = ColumnPruner(columns=[idf.getOutputCol(), hashingTF.getOutputCol(), stopWordsRemover.getOutputCol(), regexTokenizer.getOutputCol()])

##  Create the pipeline by defining all the stages
pipeline = Pipeline(stages=[regexTokenizer, stopWordsRemover, hashingTF, idf, gbm, colPruner])

## Train the pipeline model
model = pipeline.fit(sentimentDF)

model


In [45]:
model.transform(sentimentDF).take(10)


[Row(Score=5, Month=1, Day=16, WeekDay=u'Fri', Summary=u'Good Quality Dog Food', value=4.054493898772716),
 Row(Score=1, Month=1, Day=16, WeekDay=u'Fri', Summary=u'Not as Advertised', value=4.054493898772716),
 Row(Score=4, Month=1, Day=15, WeekDay=u'Thu', Summary=u'""Delight"" says it a', value=4.054493898772716),
 Row(Score=2, Month=1, Day=16, WeekDay=u'Fri', Summary=u'Cough Medicine', value=4.054493898772716),
 Row(Score=5, Month=1, Day=16, WeekDay=u'Fri', Summary=u'Great taffy', value=4.613358328249387),
 Row(Score=4, Month=1, Day=16, WeekDay=u'Fri', Summary=u'Nice Taffy', value=4.054493898772716),
 Row(Score=5, Month=1, Day=16, WeekDay=u'Fri', Summary=u'Great!  Just as good as the expensive brands!', value=4.18077098095041),
 Row(Score=5, Month=1, Day=16, WeekDay=u'Fri', Summary=u'Wonderful, tasty taffy', value=4.502975996597751),
 Row(Score=5, Month=1, Day=16, WeekDay=u'Fri', Summary=u'Yay Barley', value=4.054493898772716),
 Row(Score=5, Month=1, Day=16, WeekDay=u'Fri', Summary=u

In [43]:
sentimentDF.head()

Row(Score=5, Month=1, Day=16, WeekDay=u'Fri', Summary=u'Good Quality Dog Food')